# Things to be done

* Missing Values and its imputations
* Temporal Variables
* Trend
* Seasonality
* Moving Average

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f

In [2]:
# Create a pyspark session
spark = SparkSession.builder.config("spark.driver.memory","4g").config("spark.driver.maxResultSize", "4g").getOrCreate()
spark

22/04/06 13:53:51 WARN Utils: Your hostname, kashvi-Inspiron-5579 resolves to a loopback address: 127.0.1.1; using 10.1.72.165 instead (on interface wlp1s0)
22/04/06 13:53:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/06 13:53:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
SEED_VALUE = 42
colors = ['r', 'g', 'b'] # Seaborn color coding

In [4]:
%%time

# importing data
X_train = spark.read.csv("./Dataset/train.csv", header=True, inferSchema=True)
X_test = spark.read.csv("./Dataset/example_test.csv", header=True, inferSchema=True)
X_train.printSchema()

root
 |-- timestamp: integer (nullable = true)
 |-- Asset_ID: integer (nullable = true)
 |-- Count: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- VWAP: string (nullable = true)
 |-- Target: double (nullable = true)

CPU times: user 24 ms, sys: 4.04 ms, total: 28 ms
Wall time: 39 s


In [5]:
# Convert VWAP into double datatype
from pyspark.sql.types import FloatType, DoubleType

X_train = X_train.withColumn("VWAP", X_train.VWAP.cast(DoubleType()))
X_train = X_train.withColumn("Target", X_train.VWAP.cast(DoubleType()))
X_train.printSchema()

root
 |-- timestamp: integer (nullable = true)
 |-- Asset_ID: integer (nullable = true)
 |-- Count: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- VWAP: double (nullable = true)
 |-- Target: double (nullable = true)



In [9]:
%%time

# Count Null Values in each column
X_train.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in X_train.columns]).toPandas().head()

CPU times: user 36.1 ms, sys: 5.49 ms, total: 41.6 ms
Wall time: 28.8 s


,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,0,0,0,0,0,0,0,0,9,9


In [11]:
%%time

# Impute the null/missing values using pyspark mean functionality
missing_value_features = ["VWAP", "Target"]

for column in missing_value_features:
    mean = X_train.select(f.mean(f.col(column)).alias('mean')).collect()
    X_train.na.fill(value=mean, subset=column)
#X_train = X_train.na.fill(value=0)

TypeError: value should be a float, int, string, bool or dict

In [ ]:
def fill_with_mean(X_train, )

In [10]:
%%time

# Count Null Values in each column
X_train.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in X_train.columns]).toPandas().head()

CPU times: user 30.6 ms, sys: 12.2 ms, total: 42.8 ms
Wall time: 29.7 s


,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,0,0,0,0,0,0,0,0,9,9


In [15]:
X_train.describe().toPandas().head()

,summary,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,count,24236806,24236806,24236806,24236806,24236806,24236806,24236806,24236806,24236806,24236806
1,mean,1.5771204253124714E9,6.29254353894651,286.4593481500822,1432.6404916174413,1436.3503906875424,1429.5675986148317,1432.6399774617041,286852.985609871,NaN,NaN
2,stddev,3.323350174207381E7,4.091860643777709,867.3981577153907,6029.60501991654,6039.4824187848,6020.26068761912,6029.610646722442,2433934.645922277,NaN,NaN
3,min,1514764860,0,1.0,0.0011703999999999998,0.0011949999999999999,2.0E-4,0.0011714,-0.36628118,-Infinity,-Infinity
4,max,1632182400,13,165016.0,64805.94375000001,64900.0,64670.53,64808.537275,7.597554031423857E8,Infinity,Infinity
